## Atividades Desenvolvidas no Notebook - 02_ingestão_camada_bronze

|Sequência|Ação|Detalhamento
|---|---|---|
|SEQ-01 / SEQ-02|Ingestão dos Dados|Identificar no storage "dbfs:/FileStore/tables/landing_zone" os dados a serem inseridos na camada BRONZE.|

In [0]:
diretorio_landing_zone = "dbfs:/FileStore/tables/landing_zone/"
arquivos_diretorio = dbutils.fs.ls(diretorio_landing_zone)

In [0]:
arquivos_diretorio

Out[2]: [FileInfo(path='dbfs:/FileStore/tables/landing_zone/tb_categoria_produto/', name='tb_categoria_produto/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/landing_zone/tb_cliente/', name='tb_cliente/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/landing_zone/tb_forma_pagamento/', name='tb_forma_pagamento/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/landing_zone/tb_item_venda/', name='tb_item_venda/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/landing_zone/tb_loja/', name='tb_loja/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/landing_zone/tb_produto/', name='tb_produto/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/landing_zone/tb_venda/', name='tb_venda/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/landing_zone/tb_vendedor/', name='tb_vendedor/', size=0, modificationTime=0)]

In [0]:

for arquivo in arquivos_diretorio:

    diretorio = arquivo.path
    tabela = arquivo.name.replace('/','').lower()

    parDF1=spark.read.parquet(diretorio)

    colunas = parDF1.columns
    lista1 = ''
    listaInsert = ''
    listaCondicao = ''

    for coluna in colunas:
        if lista1 == '':
            lista1 = coluna
        else:
            lista1 = lista1 + ',' + coluna

    for coluna in colunas:
        if listaInsert == '':
            listaInsert = 'b.'+coluna
        else:
            listaInsert = listaInsert + ',' + 'b.'+ coluna

    for coluna in colunas:
        if listaCondicao == '':
            listaCondicao =  'a.'+coluna + '= b.'+coluna

    if tabela == 'tb_item_venda':
        listaCondicao = listaCondicao + ' AND a.CODIGO_PRODUTO = b.CODIGO_PRODUTO'


    tabela_dl = tabela.replace('dbo.','')

    parDF1.createOrReplaceTempView(tabela)

    comando_truncate = f"""TRUNCATE TABLE bronze.{tabela}"""

    print(comando_truncate)

    spark.sql(comando_truncate)

    comando_insert = f"""MERGE INTO bronze.{tabela} a
            USING {tabela} b
            ON {listaCondicao}
            WHEN NOT MATCHED
            THEN INSERT ({lista1}) VALUES ({listaInsert})
            """

    print(comando_insert)

    spark.sql(comando_insert)


TRUNCATE TABLE bronze.tb_categoria_produto
MERGE INTO bronze.tb_categoria_produto a
            USING tb_categoria_produto b
            ON a.CODIGO_CATEGORIA= b.CODIGO_CATEGORIA
            WHEN NOT MATCHED
            THEN INSERT (CODIGO_CATEGORIA,CATEGORIA_PRODUTO,SUB_CATEGORIA_PRODUTO) VALUES (b.CODIGO_CATEGORIA,b.CATEGORIA_PRODUTO,b.SUB_CATEGORIA_PRODUTO)
            
TRUNCATE TABLE bronze.tb_cliente
MERGE INTO bronze.tb_cliente a
            USING tb_cliente b
            ON a.CODIGO_CLIENTE= b.CODIGO_CLIENTE
            WHEN NOT MATCHED
            THEN INSERT (CODIGO_CLIENTE,NOME_CLIENTE,DATA_NASCIMENTO,IDADE_CLIENTE,DESCRICAO_ENDERECO,NUMERO_ENDERECO,NOME_CIDADE,NUMERO_TELEFONE,EMAIL_CLIENTE,OCUPACAO_CLIENTE) VALUES (b.CODIGO_CLIENTE,b.NOME_CLIENTE,b.DATA_NASCIMENTO,b.IDADE_CLIENTE,b.DESCRICAO_ENDERECO,b.NUMERO_ENDERECO,b.NOME_CIDADE,b.NUMERO_TELEFONE,b.EMAIL_CLIENTE,b.OCUPACAO_CLIENTE)
            
TRUNCATE TABLE bronze.tb_forma_pagamento
MERGE INTO bronze.tb_forma_pagamento 